In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# user-model import
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my_playground/models')

In [ ]:
from google.colab import files # 데이터 불러오기
file_uploaded=files.upload()   # chap06/data/dogs-vs-cats.zip 데이터 불러오기

In [4]:
!unzip dogs-vs-cats.zip -d dogs-vs-cats/  #dogs-vs-cats 폴더 만들어 압축 풀기

Archive:  dogs-vs-cats.zip
   creating: dogs-vs-cats/Cat/
  inflating: dogs-vs-cats/Cat/cat.0.jpg  
  inflating: dogs-vs-cats/Cat/cat.1.jpg  
  inflating: dogs-vs-cats/Cat/cat.10.jpg  
  inflating: dogs-vs-cats/Cat/cat.100.jpg  
  inflating: dogs-vs-cats/Cat/cat.101.jpg  
  inflating: dogs-vs-cats/Cat/cat.102.jpg  
  inflating: dogs-vs-cats/Cat/cat.103.jpg  
  inflating: dogs-vs-cats/Cat/cat.104.jpg  
  inflating: dogs-vs-cats/Cat/cat.105.jpg  
  inflating: dogs-vs-cats/Cat/cat.106.jpg  
  inflating: dogs-vs-cats/Cat/cat.107.jpg  
  inflating: dogs-vs-cats/Cat/cat.108.jpg  
  inflating: dogs-vs-cats/Cat/cat.109.jpg  
  inflating: dogs-vs-cats/Cat/cat.11.jpg  
  inflating: dogs-vs-cats/Cat/cat.110.jpg  
  inflating: dogs-vs-cats/Cat/cat.111.jpg  
  inflating: dogs-vs-cats/Cat/cat.112.jpg  
  inflating: dogs-vs-cats/Cat/cat.113.jpg  
  inflating: dogs-vs-cats/Cat/cat.114.jpg  
  inflating: dogs-vs-cats/Cat/cat.115.jpg  
  inflating: dogs-vs-cats/Cat/cat.116.jpg  
  inflating: dogs-vs-cat

## Library

In [3]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import pandas as pd
import LeNet5, AlexNet, VGGNet, ResNet, Pretrained
from collections import namedtuple
from sklearn.metrics import accuracy_score

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, mode):
        if mode == "test": #test 도 val과 같이 처리
            mode = "val"
        return self.data_transform[mode](img)

In [6]:
class DogvsCatDataset(Dataset):
    def __init__(self, file_list, transform=None, mode='train'):
        self.file_list = file_list
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.mode)
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0
        if self.mode == "test":
            return img_transformed, label # 원래 label을 빼야 한다
        else:
            return img_transformed, label

In [7]:
class Trainer:
    BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/my_playground"

    def __init__(self, model_name = "LeNet5", size=224):
        self.cat_path = "dogs-vs-cats/Cat/"
        self.dog_path = "dogs-vs-cats/Dog/"
        self.model_name = model_name
        self.save_path = f"{Trainer.BASE_PATH}/models_param/best_{model_name}_model.pth"
        self.vgg11_config = [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M']


        ResNetConfig = namedtuple("ResNetConfig", ['block','n_blocks','channels'])
        self.resnet50_config = ResNetConfig(block=ResNet.Bottleneck,
                                    n_blocks =[3,4,6,3],
                                    channels = [64,128,256,512])
        self.size = size
        self.mean = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

        self.BATCH_SIZE = 32
        self.NUM_WORKERS = 0
        self.EPOCH = 10
        self.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.SEED = 42
        seed_everything(self.SEED)

    def setup(self):
        """ setup """
        cat_images_filepaths = sorted([os.path.join(self.cat_path, f) for f in os.listdir(self.cat_path)])
        dog_images_filepaths = sorted([os.path.join(self.dog_path, f) for f in os.listdir(self.dog_path)])
        images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
        correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]
        random.shuffle(correct_images_filepaths)

        train_images_filepaths = correct_images_filepaths[:400]
        val_images_filepaths = correct_images_filepaths[400:-10]
        test_images_filepaths = correct_images_filepaths[-10:]

        train_dataset = DogvsCatDataset(train_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='train')
        val_dataset = DogvsCatDataset(val_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='val')
        test_dataset = DogvsCatDataset(test_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='test')

        self.train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=self.BATCH_SIZE,
            shuffle=True,
            num_workers=self.NUM_WORKERS,
            pin_memory=True
        )
        self.val_dataloader = DataLoader(
                    dataset=val_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )
        self.test_dataloader = DataLoader(
                    dataset=test_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )

        self.model_dict = {"LeNet5": LeNet5.LeNet_5(),
                           "AlexNet": AlexNet.AlexNet(),
                           "AlexNet_Pre": Pretrained.AlexNet_pre(),
                           "VGGNet": VGGNet.VGGNet(self.vgg11_config),
                           "VGGNet_Pre": Pretrained.VGGNet_pre(),
                           "ResNet": ResNet.ResNet(self.resnet50_config, output_dim=2),
                           "ResNet_Pre": Pretrained.ResNet_pre()} # 점차 쌓아 나감.
        self.model = self.model_dict[self.model_name]
        self.loss_fn = nn.CrossEntropyLoss() #criterion
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)

    def train(self):
        self.model.to(self.DEVICE)
        self.loss_fn.to(self.DEVICE)
        best_acc = 0.0

        for epoch in range(self.EPOCH):
            self.model.train()
            print('Epoch {}/{}'.format(epoch + 1, self.EPOCH))
            print('-'*20)
            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(self.train_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(self.train_dataloader.dataset)
            epoch_acc = epoch_corrects.double() / len(self.train_dataloader.dataset)
            print('[Train] Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)) #train은 필요 없을 것 같다.

            val_loss, val_acc = self.valid()
            print('[Val] Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

            if val_acc > best_acc:
                best_acc = val_acc
                best_model = self.model.state_dict()
                torch.save(self.model.state_dict(), self.save_path)

    def valid(self):
        self.model.eval()
        val_loss, val_corrects = 0.0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(self.val_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

            val_loss = val_loss / len(self.val_dataloader.dataset)
            val_acc = val_corrects.double() / len(self.val_dataloader.dataset)
        return val_loss, val_acc

    def test(self):
        model = self.model_dict[self.model_name].to(self.DEVICE)
        model.load_state_dict(torch.load(self.save_path))
        model.eval()
        id_list, pred_list = [] , []
        _id=0
        with torch.no_grad():
            for inputs, labels in tqdm(self.test_dataloader):
                for img,label in zip(inputs,labels):
                    img = img.unsqueeze(0) #왜 test 만 unsqueeze? ->  input차원을 맞춰주기 위해서
                    img = img.to(self.DEVICE)
                    outputs = model(img)
                    preds = F.softmax(outputs, dim=1)[:, 1].tolist()[0]
                    if preds > 0.5:
                        preds = 1 # dog
                    else:
                        preds = 0 # cat
                    id_list.append(label.item())
                    pred_list.append(preds)

        res = pd.DataFrame({
            'result': id_list,
            'predict': pred_list
        })

        res.to_csv(f'{Trainer.BASE_PATH}/models_result/{self.model_name}.csv', index=False)
        print(f"[Accuracy]: {accuracy_score(id_list,pred_list)}")
        # return res


# LeNet5

In [9]:
lenet_trainer = Trainer(model_name = "LeNet5",size=224)
lenet_trainer.setup()

In [10]:
lenet_trainer.train()

Epoch 1/10
--------------------


<ipython-input-8-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6935 Acc: 0.5200


<ipython-input-8-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.6880 Acc: 0.5326
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6772 Acc: 0.5875


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6864 Acc: 0.5326


In [11]:
lenet_trainer.test()

<ipython-input-8-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.5


# AlexNet

In [11]:
alex_trainer = Trainer(model_name = "AlexNet",size=227)
alex_trainer.setup()

In [12]:
alex_trainer.model

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvg

In [ ]:
alex_trainer.train()

In [15]:
alex_trainer.test()

<ipython-input-8-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.3


In [8]:
alex_pre_trainer = Trainer(model_name = "AlexNet_Pre",size=224)
alex_pre_trainer.setup()

In [9]:
alex_pre_trainer.model

AlexNet_pre(
  (alexnet_pre): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)

In [10]:
alex_pre_trainer.train()

Epoch 1/10
--------------------


<ipython-input-7-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6934 Acc: 0.4975


<ipython-input-7-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.6930 Acc: 0.5326
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6930 Acc: 0.5400


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6930 Acc: 0.5978


In [11]:
alex_pre_trainer.test()

<ipython-input-7-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.3


# VGGNet

In [8]:
vgg11_trainer = Trainer(model_name = "VGGNet",size=224)
vgg11_trainer.setup()

In [9]:
vgg11_trainer.train()

Epoch 1/10
--------------------


<ipython-input-7-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7098 Acc: 0.5200


<ipython-input-7-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.4935 Acc: 0.7717
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.5105 Acc: 0.7450


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.5454 Acc: 0.6957


In [10]:
vgg11_trainer.test()

<ipython-input-7-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.4


In [11]:
vgg11_pre_trainer = Trainer(model_name = "VGGNet_Pre",size=224)
vgg11_pre_trainer.setup()

In [12]:
vgg11_pre_trainer.train()

Epoch 1/10
--------------------


<ipython-input-7-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6952 Acc: 0.4775


<ipython-input-7-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.6909 Acc: 0.5326
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6874 Acc: 0.5650


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6913 Acc: 0.5326


In [13]:
vgg11_pre_trainer.test()

<ipython-input-7-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.5


# ResNet50

In [14]:
resnet_trainer = Trainer(model_name = "ResNet",size=224)
resnet_trainer.setup()

In [15]:
resnet_trainer.train()

Epoch 1/10
--------------------


<ipython-input-7-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7162 Acc: 0.5625


<ipython-input-7-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.8397 Acc: 0.6413
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6879 Acc: 0.6275


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6262 Acc: 0.6522


In [16]:
resnet_trainer.test()

<ipython-input-7-3a8d5f5d3a61>:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

[Accuracy]: 0.6


In [18]:
resnet_pre_trainer = Trainer(model_name = "ResNet_Pre",size=224)
resnet_pre_trainer.setup()

In [ ]:
resnet_pre_trainer.train()

Epoch 1/10
--------------------


<ipython-input-7-3a8d5f5d3a61>:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7010 Acc: 0.4800


<ipython-input-7-3a8d5f5d3a61>:115: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7814 Acc: 0.4700


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6876 Acc: 0.5326
Epoch 3/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
resnet_pre_trainer.test()